In [45]:
from sqlalchemy import select, func, and_, or_, Integer
from sqlalchemy.orm import selectinload

from models import WorkersOrm, ResumesOrm, Workload
from schemas import ResumesDTO, ResumesRelDTO, WorkersDTO, WorkersRelDTO
from database import session_factory, sync_engine

from pydantic import BaseModel

#### Без relationship

In [48]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

result_orm=[<WorkersOrm id=1, username=Jack>, <WorkersOrm id=2, username=Michael>]
result_dto=[WorkersDTO(username='Jack', id=1), WorkersDTO(username='Michael', id=2)]


#### C relationship

In [49]:
with session_factory() as session:
    query = (
        select(WorkersOrm)
        .options(selectinload(WorkersOrm.resumes))
        .limit(2)
    )

    res = session.execute(query)
    result_orm = res.scalars().all()
    print(f"{result_orm=}")
    result_dto = [WorkersRelDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

result_orm=[<WorkersOrm id=1, username=Jack>, <WorkersOrm id=2, username=Michael>]
result_dto=[WorkersRelDTO(username='Jack', id=1, resumes=[ResumesDTO(title='Python Junior Developer', compensation=50000, workload=<Workload.fulltime: 'fulltime'>, worker_id=1, id=1, created_at=datetime.datetime(2023, 10, 29, 10, 41, 12, 770381), updated_at=datetime.datetime(2023, 10, 29, 10, 41, 12, 770381)), ResumesDTO(title='Python Разработчик', compensation=150000, workload=<Workload.fulltime: 'fulltime'>, worker_id=1, id=2, created_at=datetime.datetime(2023, 10, 29, 10, 41, 12, 770381), updated_at=datetime.datetime(2023, 10, 29, 10, 41, 12, 770381))]), WorkersRelDTO(username='Michael', id=2, resumes=[ResumesDTO(title='Python Data Engineer', compensation=250000, workload=<Workload.parttime: 'parttime'>, worker_id=2, id=3, created_at=datetime.datetime(2023, 10, 29, 10, 41, 12, 770381), updated_at=datetime.datetime(2023, 10, 29, 10, 41, 12, 770381)), ResumesDTO(title='Data Scientist', compensation=3000

#### JOIN

In [52]:
class WorkloadAvgCompensationDTO(BaseModel):
    workload: Workload
    avg_compensation: int

In [53]:
with session_factory() as session:
    query = (
        select(
            ResumesOrm.workload,
            func.avg(ResumesOrm.compensation).cast(Integer).label("avg_compensation"),
        )
        .select_from(ResumesOrm)
        .filter(and_(
            ResumesOrm.title.contains("Python"),
            ResumesOrm.compensation > 40000,
        ))
        .group_by(ResumesOrm.workload)
        .having(func.avg(ResumesOrm.compensation) > 70000)
    )
    res = session.execute(query)
    result_orm = res.all()
    print(f"{result_orm=}")
    result_dto = [WorkloadAvgCompensationDTO.model_validate(row, from_attributes=True) for row in result_orm]
    print(f"{result_dto=}")

result_orm=[(<Workload.parttime: 'parttime'>, 165000), (<Workload.fulltime: 'fulltime'>, 90000)]
result_dto=[WorkloadAvgCompensationDTO(workload=<Workload.parttime: 'parttime'>, avg_compensation=165000), WorkloadAvgCompensationDTO(workload=<Workload.fulltime: 'fulltime'>, avg_compensation=90000)]
